# Notebook 2: Fine-Tuning RoBERTa

## Overview
This notebook fine-tunes a distilroberta-base model for greenwashing detection.

## Process
1. Load synthetic training and evaluation data
2. Evaluate BASELINE (untrained model) performance
3. Fine-tune the model
4. Evaluate FINE-TUNED model performance
5. Compare baseline vs fine-tuned results
6. Analyze errors

## Requirements Met
- Apply model to dataset
- Calculate metrics (Accuracy, Precision, Recall, F1)
- Fine-tune model
- Demonstrate improvement over baseline

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Libraries loaded successfully.")

## Configuration

In [ ]:
# Model and data paths
MODEL_NAME = "distilroberta-base"
TRAIN_PATH = "../inputs/train_synthetic.csv"
EVAL_PATH = "../inputs/eval_synthetic.csv"
OUTPUT_DIR = "../models/gw_finetuned"

# Training hyperparameters
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5

print(f"Model: {MODEL_NAME}")
print(f"Training for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE}")

## Load Data

In [ ]:
# Load CSV files
train_df = pd.read_csv(TRAIN_PATH)
eval_df = pd.read_csv(EVAL_PATH)

print(f"Training set: {len(train_df)} samples")
print(f"Evaluation set: {len(eval_df)} samples")
print(f"\nClass distribution in training:")
print(train_df['label'].value_counts())

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
eval_dataset = Dataset.from_pandas(eval_df[['label', 'label']])

print("\nDatasets loaded successfully.")

## Tokenization

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    """Tokenize text with padding and truncation."""
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Apply tokenization to both datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Create a dictionary for easier access
tokenized_datasets = {
    "train": tokenized_train,
    "test": tokenized_eval
}

print("Tokenization complete.")

## Model Setup

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

# Define compute metrics function
def compute_metrics(eval_pred):
    """Calculate accuracy from predictions."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Training arguments
args = TrainingArguments(
    output_dir="../models/checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=10,
    use_cpu=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

print("Trainer initialized.")

## Baseline Evaluation (Untrained Model)

Evaluate the model BEFORE training to establish a baseline.

In [ ]:
print("="*60)
print("BASELINE EVALUATION - UNTRAINED MODEL")
print("="*60)

# Evaluate untrained model
baseline_results = trainer.evaluate()
baseline_preds = trainer.predict(tokenized_datasets["test"])
baseline_pred_labels = baseline_preds.predictions.argmax(-1)
baseline_true_labels = baseline_preds.label_ids

# Store baseline accuracy for comparison
baseline_acc = baseline_results['eval_accuracy']

print(f"\nBaseline Accuracy: {baseline_acc:.4f}")
print(f"Baseline Loss: {baseline_results['eval_loss']:.4f}")

print("\nClassification Report (Baseline):")
print(classification_report(
    baseline_true_labels,
    baseline_pred_labels,
    target_names=["Vague", "Specific"],
    digits=4
))

print("\nConfusion Matrix (Baseline):")
print(confusion_matrix(baseline_true_labels, baseline_pred_labels))
print("[[TN FP]")
print(" [FN TP]]")

## Fine-Tuning

Train the model on our synthetic dataset.

In [ ]:
print("\n" + "="*60)
print("STARTING FINE-TUNING")
print("="*60)

# Train the model
trainer.train()

print("\nFine-tuning complete.")

## Fine-Tuned Model Evaluation

In [ ]:
print("\n" + "="*60)
print("FINE-TUNED MODEL EVALUATION")
print("="*60)

# Evaluate fine-tuned model
final_results = trainer.evaluate()
final_preds = trainer.predict(tokenized_datasets["test"])
final_pred_labels = final_preds.predictions.argmax(-1)
final_true_labels = final_preds.label_ids

print(f"\nFine-tuned Accuracy: {final_results['eval_accuracy']:.4f}")
print(f"Fine-tuned Loss: {final_results['eval_loss']:.4f}")

print("\nClassification Report (Fine-tuned):")
print(classification_report(
    final_true_labels,
    final_pred_labels,
    target_names=["Vague", "Specific"],
    digits=4
))

print("\nConfusion Matrix (Fine-tuned):")
print(confusion_matrix(final_true_labels, final_pred_labels))
print("[[TN FP]")
print(" [FN TP]]")

## Improvement Analysis

Compare baseline vs fine-tuned performance.

In [ ]:
print("\n" + "="*60)
print("IMPROVEMENT ANALYSIS: Baseline vs Fine-tuned")
print("="*60)

improvement = final_results['eval_accuracy'] - baseline_acc
improvement_pct = improvement * 100

print(f"\nBaseline Accuracy:     {baseline_acc:.4f}")
print(f"Fine-tuned Accuracy:   {final_results['eval_accuracy']:.4f}")
print(f"\nAbsolute Improvement:  +{improvement:.4f}")
print(f"Relative Improvement:  +{improvement_pct:.2f}%")

if improvement > 0:
    print("\nRESULT: Fine-tuning successfully improved model performance.")
else:
    print("\nWARNING: No improvement observed.")

## Error Analysis

Examine misclassified examples to understand model limitations.

In [ ]:
print("\n" + "="*60)
print("ERROR ANALYSIS")
print("="*60)

# Find misclassifications
errors = []
for i, (pred, true) in enumerate(zip(final_pred_labels, final_true_labels)):
    if pred != true:
        errors.append({
            "text": eval_df.iloc[i]["text"],
            "predicted": "Specific" if pred == 1 else "Vague",
            "actual": "Specific" if true == 1 else "Vague"
        })

print(f"\nTotal Misclassifications: {len(errors)} / {len(final_true_labels)}")
print(f"Error Rate: {len(errors)/len(final_true_labels)*100:.2f}%")

if errors:
    print("\nExample Misclassifications:")
    print("-" * 60)
    for i, err in enumerate(errors[:5], 1):
        print(f"\nError {i}:")
        print(f"Text: {err['text'][:150]}..." if len(err['text']) > 150 else f"Text: {err['text']}")
        print(f"Predicted: {err['predicted']} | Actual: {err['actual']}")
        print("-" * 60)
else:
    print("\nNo errors found - perfect classification on evaluation set.")
    print("Note: This may indicate overfitting on synthetic data.")

## Save Model

In [ ]:
# Save the fine-tuned model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"\nFine-tuned model saved to: {OUTPUT_DIR}")
print("\nModel is ready for use in Notebook 3: Greenwashing Analysis")

## Summary

This notebook demonstrated:
1. Baseline evaluation of untrained model
2. Fine-tuning on synthetic greenwashing data
3. Comprehensive metrics (Accuracy, Precision, Recall, F1)
4. Quantified improvement over baseline
5. Error analysis of misclassifications

The model is now ready to classify real corporate sustainability reports.